#  支持向量机（Support Vector Machine）

支持向量机SVM，是一种**分类算法**，它的特点在于**找出的分类面是最优的**。对于二维数据来说，它可以找到一条直线，这是无数条可以分类的直线当中最完美的，因为它恰好在两个类的中间。如果是高维的点，SVM的分界线就是平面（plane）或者超平面（hyperplane）。假设以二维数据的分类为例，其算法结构可由下图（图片来源[1]）所展示：

![svm1](./resources/svm1.jpg)

上图中：**Boundary代表决策面**；**Margin代表分类间隔**；**Support Vector代表离决策面最近的那些数据点（虚线穿过的点），即支持向量点**。

一些需要注意的地方：

- 如果一个类别的数据点跑到了另一类别中，那么它就是该类别的离散值。SVM 的一个特征就是会忽略离散值并找到具有最大边距的超平面。因此，我们可以说，**SVM对于离散值具有鲁棒性**。
- SVM有核函数，该函数具有**将低维数据转化成高维数据的作用**，它将**低维不可分**的问题转换成了**高维可分**的问题。
- 实际上**最优决策面的方向和位置完全取决于选择哪些样本点作为支持向量**。

## 求解最优决策面

一个最优化问题通常有两个最基本的因素：

- **目标函数**，也就是你希望什么东西的什么指标达到最好;
- **优化对象**，你期望通过改变哪些因素来使你的目标函数达到最优;

在线性SVM算法中，目标函数是**分类间隔**，而优化对象则是**决策面**。

### 直接表达形式

求解最优决策面最直接的表达形式（Direct Representation）如下所示，其中$margin$函数可以输入一个$boundary$，计算正确分类的所有苹果与香蕉到$boundary$的最小距离。

> $argmax_{boundary} \ margin(boundary)$  
> $s.t.$ 所有正确归类的苹果与香蕉到$boundary$的距离都$>=margin$

### 数学表达形式

1.定义直线$x_2=ax_1+b$为决策面，经过变换可以得到：$[a,-1]\left[ \begin{array}{c}x_1\\x_2\end{array} \right] +b=0，再写成：$$\omega^Tx+\gamma=0$，可以证明$\omega$控制了直线的方向。

2.平面上任意点$x_0$到直线的距离为：$\dfrac{1}{||\boldsymbol{\omega}||}(\boldsymbol{\omega}^Tx_0+\gamma)$，假设支持向量到决策面的距离为$d$，则所有样本点需要满足：

> $\left\{\begin{array}{ll} (\boldsymbol{\omega}^T\boldsymbol{x}_i+\gamma)/||\boldsymbol{\omega}||\geq d & \textrm{for} \ y_i=1\\(\boldsymbol{\omega}^T\boldsymbol{x}_i+\gamma)/||\boldsymbol{\omega}||\leq -d & \textrm{for} \ y_i=-1\end{array}\right. $

两边同除以$d$不影响原决策面的方向与截距，则SVM优化的约束条件变为：

> $\left\{\begin{array}{ll} (\boldsymbol{\omega}^T\boldsymbol{x}_i+\gamma)/(||\boldsymbol{\omega}||\times d)\geq 1 & \textrm{for} \ y_i=1\\(\boldsymbol{\omega}^T\boldsymbol{x}_i+\gamma)/(||\boldsymbol{\omega}||\times d)\leq -1 & \textrm{for} \ y_i=-1\end{array}\right. $

因为$(||\boldsymbol{\omega}||\times d)$是常量不影响直线的方向，所以SVM优化的约束条件变为：

> $\left\{\begin{array}{ll} \boldsymbol{\omega}^T\boldsymbol{x}_i+\gamma\geq 1 & \textrm{for} \  y_i=1\\\boldsymbol{\omega}^T\boldsymbol{x}_i+\gamma\leq -1 & \textrm{for} \ y_i=-1\end{array}\right. $

当$x_i$为支持向量点的时候，$\boldsymbol{\omega}^T\boldsymbol{x}_i+\gamma$等于1或者-1，这个时候$d$等于：

> $d = \dfrac{|\boldsymbol{\omega}^T\boldsymbol{x}_i+\gamma|}{||\boldsymbol{\omega}||}=\dfrac{1}{||\boldsymbol{\omega}||},~~\textrm{if} ~\boldsymbol{x}_i \textrm {is a support vector} $

则问题转化为求$||\boldsymbol{\omega}||$最小值的问题。综上，SVM最优化问题的数学描述最终变为：

> $\begin{array}{l} \min_{\boldsymbol{\omega},\gamma}\dfrac{1}{2}||\boldsymbol{\omega}||^2\\ \textrm{s. t.}~ ~y_i(\boldsymbol{\omega}^T\boldsymbol{x}_i+\gamma)\geq 1,~~i = 1,2,...,m \end{array}$

缩写$s.t.$表示“Subject to”，是“服从某某条件”的意思，之所以要在$||\boldsymbol{\omega}||$上加上平方和1/2的系数，是为了以后进行最优化的过程中对目标函数求导时比较方便,但这绝不影响最优化问题最后的解。于是，求解SVM最优决策面的问题转化成了一个**不等式约束条件下的优化问题**。

### 求解不等式约束的SVM最优化问题

其拉格朗日函数可以写为：

$L(\boldsymbol{\omega},\gamma,\boldsymbol{\alpha})=\dfrac{||\boldsymbol{\omega}||^2}{2}+\sum_{i=1}^m\alpha_i(1-y_i(\boldsymbol{\omega}^T\boldsymbol{x}_i+\gamma))$  

该拉格朗日函数最优化的原始问题和对偶问题分别为：

>  $\min_{\boldsymbol{\omega},\gamma}\left[ \max_{\boldsymbol{\alpha}:\alpha_j\geq0}L(\boldsymbol{\omega},\gamma,\boldsymbol{\alpha}) \right] $  
> $\max_{\boldsymbol{\alpha}:\alpha_j\geq0}\left[ \min_{\boldsymbol{\omega},\gamma}L(\boldsymbol{\omega},\gamma,\boldsymbol{\alpha}) \right]  $

按照[不等式约束条件下最优化问题的求解](./optimal_solution_with_constraint.ipynb)的例子来计算最优值。

### 另一种理解大间隔的思路
按照吴恩达在机器学习中的思路，SVM算法的代价函数为：$min_{\theta}C\sum_1^m[y^{(i)}cost_1(\theta^Tx^{(i)}) + (1-y^{(i)})cost_0(\theta^Tx^{(i)})] + \dfrac{1}{2}\sum_i^n\theta_j^2$

代价函数中两个cost函数的图像为：

![svm6](./resources/svm6.jpg)

假设将$C$设置成一个很大的值，则希望找一个使代价函数第一项等于0的最优解（即：当$y=1$,则要求$\theta^Tx>=1$; 当$y=0$,则要求$\theta^Tx<=-1$）。则优化问题转变为：

![svm7](./resources/svm7.png)

> 注意：如果$C$设置的过大将会很受异常点影响；  
> $C$的作用等同于$1/\lambda$ (之前使用过的正则化参数);

如果将样本$x^{(i)}$与$\theta$画到一起，则$\theta^Tx^{(i)}$可以变为$p^{(i)}\cdot ||\theta||$, 如下图所示：

![svm8](./resources/svm8.png)

为了让目标函数中对应的cost函数尽量大于1或者小于-1，唯一的方法就是通过旋转找到一条$\theta$的范数相对小，但是数据点在上面的投影$p^{(i)}$又比较大的决策面。

#### 核函数
对于复杂的决策边界可能需要复杂的模型（比如高维非线性模型），这就需要更多新的特征，核函数就是一种来计算出新的特征的方法。给定一个训练实例$x$，我们利用$x$的各个特征与我们预先选定的地标(landmarks): $l^{(1)},l^{(2)},l^{(3)}$的近似程度来选取新的特征$f_1,f_2,f_3$。

![svm9](./resources/svm9.png)

例如：${{f}_{1}}=similarity(x,{{l}^{(1)}})=e^{(-\dfrac{{{\left| x-{{l}^{(1)}} \right|}^{2}}}{2{{\sigma }^{2}}})}$，这里$f_1$就是一个高斯核函数，不使用核函数又称为线性核函数(linear kernel)。

这里给出$C$与$\sigma$对支持向量机的影响：
- $C$较大时，相当于$\lambda$较小，可能会导致过拟合，高方差；
- $C$较小时，相当于$\lambda$较大，可能会导致低拟合，高偏差；
- $\sigma$较大时，可能会导致低方差，高偏差；
- $\sigma$较小时，可能会导致低偏差，高方差。

**很大训练集+高斯核函数可能会很慢。逻辑回归跟不带核函数的SVM表现很相似。SVM具有的优化问题，是一种凸优化问题。**

## 参考文档

1. [支持向量机(SVM)是什么意思？](https://www.zhihu.com/question/21094489/answer/117246987)  
2. [零基础学SVM—Support Vector Machine(一)](https://zhuanlan.zhihu.com/p/24638007)